# Exploration

This notebook is used for functional exploration, in order to figure out how things work, so that they then can be implemented in the allskycam.py-Module


In [ ]:
import allskycam

import ipywidgets as widgets
from IPython.display import display

from io import BytesIO
from time import sleep
from picamera import PiCamera

from matplotlib import pyplot as plt

import time
import numpy as np

## Camera capture to stream and display single image using PiCamera

In [ ]:
stream = BytesIO()

# grab image from camera
with PiCamera() as camera:
    camera.capture(stream, format='jpeg')
    stream.seek(0)
    jpeg_image = stream.read()

# display image
widgets.Image(
    value=jpeg_image,
    format='jpg',
    width=900
)

## Camera capture to stream and display sequence of images using PiCamera

In [ ]:
stream = BytesIO()

wgt = widgets.Image(
    format='jpeg',
    width=900
)
display(wgt)

# grab image from camera
with PiCamera() as camera:
    for i in range(100):
        camera.capture(stream, format='jpeg')
        stream.seek(0)
        jpeg_image = stream.read()
        wgt.value = jpeg_image

## Acessing and controlling the camera

It is very important to set both, framerate as well as shutter_speed to consistant values.

If the frame_rate is too high, it is not possible to reach the exposure time, if the frame_rate is too low, a lot of time is needed to finalize the image capture sequence.

In this example, the frame rate time is set 1 second longer than the exposure time.

In [ ]:
import picamera
from picamera import PiCamera
from time import sleep
from fractions import Fraction
picamera.CAPTURE_TIMEOUT = 1000
PiCamera.CAPTURE_TIMEOUT = 1000

In [ ]:
stream = BytesIO()
expt = 180

t0 = time.time()
with PiCamera( framerate=Fraction(1, expt+1), sensor_mode=3) as camera:
    allskycam.set_analog_gain(camera, 16)
    allskycam.set_digital_gain(camera, 6)
    camera.resolution = (1024, 768)
    camera.still_stats = False
    camera.shutter_speed = expt*1000000
    camera.awb_gains = (2.4, 2.21)
    camera.exposure_mode = 'off'
    camera.awb_mode = 'off'
    camera.capture(stream, format='jpeg',bayer=True)
    stream.seek(0)
    jpeg_image = stream.read()
    t1 = time.time()
print('%3.2f seconds'%(t1-t0))

In [ ]:
imw = widgets.Image(format='jpg')
imw.value = jpeg_image
display(imw)

In [ ]:
# getting the raw image data
stream.seek(0)
raw = allskycam.get_bayer_data_from_stream(stream)

R = raw[0::2,0::2]
G1 = raw[1::2,0::2]
G2 = raw[0::2,1::2]
B = raw[1::2,1::2]

R = R.flatten()
G = np.append(G1.flatten(), G2.flatten())
B = B.flatten()

Gm = np.median(G)
Rm = np.median(R)
Bm = np.median(B)

(Gm/Rm, Gm/Bm)

In [ ]:
plt.hist(R,512, color='r', histtype=u'step');
plt.hist(G,512, color='g', histtype=u'step');
plt.hist(B,512, color='b', histtype=u'step');
plt.xlim((0,600))

## Looping Image Akquisition

In the next section, the image akquisition is looped. Data is fetched from the camera including raw bayer data, for evaluation, but for storing the file, raw bayer data is striped away.


In [2]:
import picamera
from picamera import PiCamera
from time import sleep
from fractions import Fraction
picamera.CAPTURE_TIMEOUT = 240
PiCamera.CAPTURE_TIMEOUT = 240
from io import BytesIO
import time
import allskycam
import ipywidgets as widgets
from IPython.display import display

picamera.CAPTURE_TIMEOUT = 1000
PiCamera.CAPTURE_TIMEOUT = 1000

In [ ]:
stream = BytesIO()
expt = 45
period = 60

for i in range(600):
    wait_for_period(period)
    t1 = time.time()
    filename = time.strftime("%Y%m%d%H%M%S")+'.jpg'
    with PiCamera( framerate=Fraction(1, expt+1), sensor_mode=3) as camera:
        t0 = t1
        stream.seek(0)
        allskycam.set_analog_gain(camera, 8)
        allskycam.set_digital_gain(camera, 1)
        camera.resolution = (1024, 768)
        camera.still_stats = False
        camera.shutter_speed = expt*1000000
        camera.awb_gains = (2.4, 2.21)
        camera.exposure_mode = 'off'
        camera.awb_mode = 'off'
        camera.capture(stream, format='jpeg',bayer=True)
        stream.seek(0)
        jpeg_image = stream.read()
        t1 = time.time()
        if i==0:
            imw = widgets.Image(value = jpeg_image, format='jpg')
            display(imw)
        imw.value = jpeg_image
        #print('%3.2f seconds'%(t1-t0))
    stream.seek(0)
    striped = allskycam.strip_bayer_data(stream)
    with open(filename, 'wb') as f:
        f.write(striped)
        

Image(value=b'\xff\xd8\xff\xe1c\xd0Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\n\x01\x00\x00\x04\x00\x00\x00\x01\x…

In [20]:
def wait_for_period(p):
    t = time.time()
    st = (int(t/p)+1)*p-t
    time.sleep(st)

In [24]:
print(__name__)

__main__


# Save jpeg image with stripped away raw data

An image shall be akquired including raw data.
The raw data part shall be truncated away leaving behind the plain jpeg.
The truncated file without raw data shall be stored.


In [ ]:
import picamera
from picamera import PiCamera
from time import sleep
from fractions import Fraction
picamera.CAPTURE_TIMEOUT = 240
PiCamera.CAPTURE_TIMEOUT = 240
from io import BytesIO
import time
import allskycam
import ipywidgets as widgets
from IPython.display import display

picamera.CAPTURE_TIMEOUT = 1000
PiCamera.CAPTURE_TIMEOUT = 1000

stream = BytesIO()

In [ ]:
expt = 10
t0 = time.time()
with PiCamera(sensor_mode=3) as camera:
    allskycam.set_analog_gain(camera, 1)
    allskycam.set_digital_gain(camera, 1)
    camera.resolution = (1024, 768)
    camera.still_stats = False
    camera.framerate = 1/expt
    camera.shutter_speed = int(expt*1000000)
    camera.awb_gains = (2, 2.21)
    camera.exposure_mode = 'off'
    camera.awb_mode = 'off'
    camera.capture(stream, format='jpeg',bayer=True)
    t1 = time.time()
    print('%3.2f seconds for a %3.2f s exposure'%((t1-t0), camera.exposure_speed/1000000))
    print(camera.exposure_speed)
    stream.seek(0)
    jpeg_image = stream.read()

imw = widgets.Image(format='jpg')
imw.value = jpeg_image
display(imw)

In [ ]:
from picamera import PiCamera
import time

with PiCamera(sensor_mode=2) as cam:
    exposure_time = 1.0 
    cam.framerate = 1/exposure_time
    cam.iso = 100
    cam.shutter_speed = int(exposure_time*1000000)
    cam.resolution = (1600, 900)
    cam.exposure_mode = 'off'
    time.sleep(5)
    cam.capture("test.jpg")
    print(cam.exposure_speed)

# Using PiCamera[Array]-functionallity

In [ ]:
import picamera
import picamera.array

with picamera.PiCamera() as camera:
    with picamera.array.PiBayerArray(camera) as output:
        camera.capture(output, 'jpeg', bayer=True)
        print(output.array.shape)